<a href="https://colab.research.google.com/github/luciainnocenti/IncrementalLearning/blob/daRunnareFrancy/ICaRLMain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import GitHub repository

In [1]:
import os
import logging
import sys

In [2]:
if not os.path.isdir('./DatasetCIFAR'):
  !git clone -b daRunnareFrancy https://github.com/luciainnocenti/IncrementalLearning.git
  !mv 'IncrementalLearning' 'DatasetCIFAR'

Cloning into 'IncrementalLearning'...
remote: Enumerating objects: 126, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 1437 (delta 75), reused 0 (delta 0), pack-reused 1311
Receiving objects: 100% (1437/1437), 946.21 KiB | 8.60 MiB/s, done.
Resolving deltas: 100% (917/917), done.


# Import packages

In [3]:
from DatasetCIFAR.data_set import Dataset 
from DatasetCIFAR.data_set import Subset
from DatasetCIFAR import ResNet
from DatasetCIFAR import utils
from DatasetCIFAR import params
from DatasetCIFAR import ICaRLModel
from torchvision import models
import torch.nn as nn
import torch
import torch.optim as optim
import torchvision
import numpy as np

from torchvision import transforms
from torch.utils.data import DataLoader
from torch.nn import functional as F
import random
random.seed(params.SEED)

In [4]:
print(params.SEED)
print(params.NUM_WORKERS)

653
4


# Define Datasets

In [5]:
train_transformer = transforms.Compose([transforms.RandomCrop(size = 32, padding=4),
                                         transforms.RandomHorizontalFlip(),
                                         transforms.ToTensor(),
                                         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_transformer = transforms.Compose([transforms.ToTensor(),
                                         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [6]:
trainDS = Dataset(train=True)
testDS = Dataset(train=False)

Extracting data/cifar-100-python.tar.gz to data
Files already downloaded and verified


In [7]:
train_splits = trainDS.splits
test_splits = testDS.splits

# Define Network

In [8]:
ICaRL = ResNet.resnet32(num_classes=100)
ICaRL =  ICaRL.to(params.DEVICE)

In [9]:
exemplars = [None]*100

test_indexes =  []

In [10]:
for task in range(0, 100, params.TASK_SIZE):
  train_indexes = trainDS.__getIndexesGroups__(task)
  test_indexes = test_indexes + testDS.__getIndexesGroups__(task)

  train_dataset = Subset(trainDS, train_indexes, transform = train_transformer)
  test_dataset = Subset(testDS, test_indexes, transform = test_transformer)

  train_loader = DataLoader( train_dataset, num_workers=params.NUM_WORKERS, batch_size=params.BATCH_SIZE, shuffle=True)
  test_loader = DataLoader( test_dataset, num_workers=params.NUM_WORKERS, batch_size=params.BATCH_SIZE , shuffle=True )

  ICaRL, exemplars = ICaRLModel.incrementalTrain(task, trainDS, ICaRL, exemplars, train_transformer)

  col = []
  for i,x in enumerate( train_splits[ :int(task/10) + 1]) : 
    v = np.array(x)
    col = np.concatenate( (col,v), axis = None)
    col = col.astype(int)
  mean = None
  total = 0.0
  running_corrects = 0.0
  for img, lbl, _ in train_loader:
      img = img.float().to(params.DEVICE)
      preds, mean = ICaRLModel.classify(img, exemplars, ICaRL, task, trainDS, mean)
      preds = preds.to(params.DEVICE)
      labels = utils.mapFunction(lbl, col).to(params.DEVICE)
      #print("preds: ", preds.data)
      #print("mapped labels: ", labels)
      #print("labels: ", lbl)
      total += len(lbl)
      running_corrects += torch.sum(preds == labels.data).data.item()
      #print(running_corrects)
  accuracy = float(running_corrects/total)
  print(f'task: {task}', f'train accuracy = {accuracy}')

  total = 0.0
  running_corrects = 0.0
  for img, lbl, _ in test_loader:
      img = img.float().to(params.DEVICE)
      preds, _ = ICaRLModel.classify(img, exemplars, ICaRL, task, trainDS, mean)
      preds = preds.to(params.DEVICE)
      labels = utils.mapFunction(lbl, col).to(params.DEVICE)
      #print("preds: ", preds.data)
      #print("mapped labels: ", labels)
      #print("labels: ", lbl)
      total += len(lbl)
      running_corrects += torch.sum(preds == labels.data).data.item()
      #print(running_corrects)
  accuracy = float(running_corrects/total)
  print(f'task: {task}', f'test accuracy = {accuracy}')

At step  0  and at epoch =  0  the loss is =  0.029667191207408905  and accuracy is =  0.185
At step  0  and at epoch =  1  the loss is =  0.028233041986823082  and accuracy is =  0.3734
At step  0  and at epoch =  2  the loss is =  0.018071454018354416  and accuracy is =  0.465
At step  0  and at epoch =  3  the loss is =  0.02255549468100071  and accuracy is =  0.5108
At step  0  and at epoch =  4  the loss is =  0.014904298819601536  and accuracy is =  0.5284
At step  0  and at epoch =  5  the loss is =  0.01924349181354046  and accuracy is =  0.59
At step  0  and at epoch =  6  the loss is =  0.010225322097539902  and accuracy is =  0.6086
At step  0  and at epoch =  7  the loss is =  0.031309131532907486  and accuracy is =  0.6466
At step  0  and at epoch =  8  the loss is =  0.025189409032464027  and accuracy is =  0.6548
At step  0  and at epoch =  9  the loss is =  0.01838390715420246  and accuracy is =  0.6782
At step  0  and at epoch =  10  the loss is =  0.015962667763233185